# Получение предсказаний детекции отходов

## Импорт необходимых библиотек

In [1]:
import os

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
import cv2

import torch
import torchvision
from torchvision.utils import draw_bounding_boxes
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.ops import box_convert

In [2]:
%pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.9/896.9 kB 15.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Настройки для инференса

In [3]:
# Выбор процессора
DEVICE = (
    "cuda"
    if torch.cuda.is_available()
    # else "mps"
    # if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {DEVICE} device")

Using cuda device


In [4]:
!nvidia-smi 

Sun Dec  1 20:44:33 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             10W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# Загрузим обученную нами модель
model = YOLO('/kaggle/input/garbage-detection/runs/detect/garbage_detection/weights/best.pt') 

In [6]:
print('Модель загружена')

Модель загружена


In [7]:
# Зададим переменные с путями к изображениям и меткам трейна
# а также путь к файлу для аутпут
test_img_path = '/kaggle/input/garbage-detection-data/yolo_dataset/images/test/'
output_csv_path = "/kaggle/working/detection_results.csv"

test_file_names = []
# Сохраним названия изображений трейна в переменую
for dirname, _, filenames in os.walk(test_img_path):
    for filename in filenames:
        test_file_names.append(os.path.join(test_img_path,filename))

In [8]:
print('Сформирован список файлов')

Сформирован список файлов


## Получение предсказаний

In [9]:
step = 5
results_list = []
for i in range(0, len(test_file_names), 5):
    # Получаем текущий батч файлов
    batch = test_file_names[i:i+step]
    # Осуществляем инференс на текущем батче
    result = model(batch, verbose=False)

    # Сохраняем результаты
    for r in result:
        result_data = {
            'boxes': r.boxes,          # Координаты детекций
            #'keypoints': r.keypoints,  # Ключевые точки (если есть)
            #'masks': r.masks,          # Маски (если есть)
            'names': r.names,          # Названия классов
            #'obb': r.obb,              # Oriented bounding boxes (если поддерживается)
            #'orig_img': r.orig_img,    # Исходное изображение
            #'orig_shape': r.orig_shape,  # Исходная форма изображения
            'path': r.path,            # Путь к изображению
            'probs': r.probs#,          # Вероятности классов
            #'save_dir': r.save_dir,    # Директория сохранения
            #'speed': r.speed           # Время выполнения (preprocess, inference, postprocess)
        }
        results_list.append(result_data)


In [10]:
# Создаем пустой список для хранения данных
csv_data = []

# Обрабатываем результаты из results_list
for res in results_list:
    file_name = res['path'].split('/')[-1]  # Извлекаем имя файла из полного пути
    bboxes = []

    if res['boxes'] is not None:  # Если объекты найдены
        for box in res['boxes'].data:  # Перебираем все боксы
            category_id = int(box[-1])+1  # Класс объекта
            confidence = float(box[-2])  # Уверенность модели
            x1, y1, x2, y2 = map(float, box[:4])  # Координаты бокса (x1, y1, x2, y2)
            
            # Конвертация в формат [x_left, y_top, width, height]
            x_left = x1
            y_top = y1
            width = x2 - x1
            height = y2 - y1

            bboxes.append([category_id, confidence, x_left, y_top, width, height])

    # Добавляем данные в список
    csv_data.append({'file_name': file_name, 'bbox': bboxes})

# Преобразуем данные в DataFrame
df = pd.DataFrame(csv_data)

# Сохраняем DataFrame в CSV
df.to_csv(output_csv_path, index=False)